The intent of this notebook is to document different types of machine learning approaches and algorithms. At this point it's a pile of everything, I expect to break this up into more modular pieces as more and more content is added.

# Supervised learning
Wikipedia has an excellent definition:

**Supervised learning** is the machine learning task of inferring a function from labeled training data.
The training data consist of a set of training examples. In supervised learning, each example is a pair consisting of an input object (typically a vector) and a desired output value (also called the supervisory signal). A supervised learning algorithm analyzes the training data and produces an inferred function, which can be used for mapping new examples. An optimal scenario will allow for the algorithm to correctly determine the class labels for unseen instances. This requires the learning algorithm to generalize from the training data to unseen situations in a "reasonable" way 

## Classification ##
### Support Vector Machines (SVMs) ###
SVMs are a set of learning algorithms for to do classification. In a nutshell, SVMs try to find a hyperplane that maximes the distance between clusters of data.

Visually:

![SVM Example](svm/svm-example.png)

The points/vectors that define where the hyperplane fits are called the **support vectors** (they're "supporting" the maximum distance between the clusters and the plane).


![Support Vectors](svm/support-vectors.png)


Computing SVMs comes down to solving a differential equation: finding a hyperplane that minimizes the distance between the hyperplane and all potential support vectors in their respective clusters. There are multiple algorithms for solving this.

![Support Vectors](svm/svm-minimize.png)

Note that a tricky part of defining an SVMs is often finding the right feature space. What might be hard in one dimension, might become easy in a higher dimension that represents the right features:

![Support Vectors](svm/input-feature-space.png)


#### SVMs in  Scikit-learn ####

In Scikit-learn, the simplest SVM that corresponds to the pictures above is ```LinearSVC```. The algorithm takes a ```C``` value which represents the amount of error/margin the SVM will allow when making calculations. The images below visualize the difference between C values (from [Hands-On Machine Learning with Scikit-Learn and TensorFlow](http://shop.oreilly.com/product/0636920052289.do))


![C value in scikit-learn](svm/svm-scikit-learn-C.png)

The question is which ```C``` value better captures the true nature relationship of the data. A ```C``` value that is too high might cause overfitting: the model might be too much tailored to the learning data and not generalize well.



#### Detecting multiple classes using SVMs ####

SVMs are binary classifiers: they can inheritly only classify 2 classes (as the found function divides the hyperplane in 2). Still, SVMs are often used to find multiple classes by training and SVM for each of the target classes and then comparing fit or loss scores. For example, when solving the classic MNIST digit classification problem using an SVM, as done in [sk-learn-intro.ipynb](../sk-learn-intro.ipynb), we train 10 SVMs, one for each digit. The SVM for digit 5 then can recognize whether a digit is either 5 or not 5, with a certain associated accuracy. When classifing a random digit, we simply run all SVMs and compare their scores and pick the one with the highest score.

Note that scikit-learn takes care of running all 10 SVMs and selecting the class with the highest score automatically for us.

This strategy of training *x* SVMs for *x* target classes, where each SVM can only distinguish between its class and *everything else* is called the *one-versus-all (OvA)* strategy. This is the default in scikit-learn. An alternative strategy, *one-versus-one (OvO)* - ```OneVsOneClassifier```, creates SVMs that can detect pairs of classes (e.g. SVM that can detect 5 or 6). A downside of this approach is that you need to create SVMs for all possible pairs. For MNIST this means a combination of 2 out of 10:

$$C(10,2) =\frac{10!}{(2!(10−2)!)} = 45$$

# Decision Trees #

Scikit-Learn uses the CART algorithm, which produces only binary trees: nonleaf nodes always have two children (i.e., questions only have yes/no answers). There are other algorithms that use produce 3 or more leafs per parent.

From [Hands-On Machine Learning with Scikit-Learn and TensorFlow, Chapter 6](http://shop.oreilly.com/product/0636920052289.do):
The idea behind the CART algorithm is quite simple: the algorithm first splits the training set in two subsets using a single feature k and a threshold tk (e.g., "petal length <= 2.45 cm"). How does it choose $k$ and $t_k$? It searches for the pair $(k, tk)$ that produces the purest subsets (weighted by their size). Purity is defined by the [Gini measure](https://en.wikipedia.org/wiki/Gini_coefficient). Once it has successfully split the training set in two, it splits the subsets using the same logic, then the sub-subsets and so on, recursively. It stops recursing once it reaches the maximum depth (defined by the max_depth hyperparameter), or if it cannot find a split that will reduce impurity. 

#  Neural Networks #

Very good general overview: https://www.youtube.com/watch?v=aircAruvnKk

Neural network example, with input, output and hidden layers:

![Neutral Network](nn/neural_net.png)

Individual **neurons** are just sum or **"transfer"** functions:

![Sum Function](nn/sum-function.png)

In most cases they have activation functions that will output constant or variable values if the output of the neuron reaches a certain threshold:

![Activation Function](nn/activation-function.png)

Note that $w_{ij}$ represents the weights for input $i$ in neuron $j$ which can be represented as a matrix.


There are multiple types of activation functions. Some popular ones:
 - [Sigmoid function](https://en.wikipedia.org/wiki/Sigmoid_function), ```σ(x)σ(x)```: squashes numbers into the range (0, 1)
 - The [rectified linear unit](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)), ```ReLU(x)=max(0,x)```. This is quickly becoming one of the more popular activation functions, especially in Deep Learning, as it turns out that it's performance is better than the sigmoid function (and the function is simpler).
 - The [hyperbolic tangent](https://en.wikipedia.org/wiki/Hyperbolic_function#Standard_analytic_expressions): ```tanh(x)```, which squashes numbers into the range (-1, 1)

#### Types ####
There are different neural network types:

![Neural Network Types](nn/neural-network-types.jpg)
    
    
**Perceptron**: no hidden layers, only input and output.

**Feed Forward**: No cycles or loops in the network.

**Deep Neural Networks**: neural networks that contain more than one hidden layer. 

**Recurrent Neural Network (RNN)**: also propagate data from later processing stages to earlier stages.
    

## Recurrent Neural Networks ##

A RNN maintains internal memories about the world (weights assigned to different pieces of information) to help perform its classifications. For example, when classifying activities in movie clips, it will "remember" what has happened in previous clips.

![Recurrent Neural Network](nn/rnn.png)


In this image, $\phi$ is the activation function, $W$ is the weights matrix associated with the current state, $U$ is the weights matrix associated with the previous state.

This can then in programming terms be interpreted as running a fixed program with certain inputs and some internal variables. 

A very simple implemtentation of an RNN might look like this (from http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [4]:
class RNN:
  # ...
  def step(self, x):
    # update the hidden state
    self.h = np.tanh(np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x))
    # compute the output vector
    y = np.dot(self.W_hy, self.h)
    return y

Note how this compares to the picture above. In particular, note the following similarilites:
- the ```np.tanh``` activation function, which squashes the output between -1 and 1
- $h_t$ from the previous is called ```self.h```
- The terms in the sum inbetween ```np.tanh(...)``` are switched here, it's basically: ```np.tanh(prev state + current state)```, while the image above does $\phi (current + prev)$
- $W$ is called ```self.W_xh```, $U$ is called ```self.W_hh```
- in math terms, the code really does: $h_t = \tanh ( W_{hh} h_{t-1} + W_{xh} x_t ))$

## Long short-term memory Networks (LTSM) ##

A lot of the info that follows is based off this blogpost: http://blog.echen.me/2017/05/30/exploring-lstms/

Whereas an RNN can overwrite its memory at each time step in a fairly uncontrolled fashion, an LSTM (specific type of RNN) transforms its memory in a very precise way: by using specific learning mechanisms for which pieces of information to remember, which to update, and which to pay attention to. This helps it keep track of information over longer periods of time.

In the code above, and LTSM would make the computation of ```self.h``` more complicated.

# Convolutional Neural Networks (CNN) #

Mostly used for visual recognition tasks (e.g. feature detection in images). Inspired by the human vision-related neural system. In CNNs, not every neuron in a layer is connected to every other neuron in the next layer. Instead neurons are only connected to *some* neurons in the next layer. In a sense, layers get aggregated into the next layer as shown in the next 2 images from [Hands-On Machine Learning with Scikit-Learn and TensorFlow](http://shop.oreilly.com/product/0636920052289.do).

![CNN](nn/cnn.png)

More detailed:

![CNN](nn/cnn-detail.png)


Note that this is different to connecting every pixel/neutron to every other pixel/neutron in the next layer (this is how traditional RNNs work). This is also how the human vision neural system works: neurons only have a limited **receptive field** (i.e. The neurons that can trigger a particular neuron are in the local neighborhood and limited to a small area). Neurons have a form of hierarchy in which some neurons that detect lower level structure (e.g.: lines) trigger neurons that recognize higher-level structure (e.g. shapes), and so on.


In a CNN, A neuron's weights can be represented as a small image the size of the receptive field. These sets of weights are also called **filters** or **convolution kernels**. 



## Pooling
(Joris: Pooling layers seem to be specific to CNNs.)

Pooling layers group/pool inputs together into a smaller layer. This reduces the computational cost by reducing the number of parameters to learn, while often still providing good results. For example, often times, not all input numbers are needed to detect a feature.

For example, in max pooling, we just take the highest (=max) number of each window we're considering. In the example below, the filter size = 2 (because the window=2x2), and the stride = 2, because we jump to steps to the right/bottom after applying the filter (i.e. there is no overlap between consecutive filter applications, which would be the case if stride was e.g. equal to 1).

![Max Pooling](nn/max-pooling.webp)

Average Pooling is sometimes also used (averaging the numbers in the window), but this isn't often used.


# Unsupervised Learning

With unsupervised learning, there are no output values or labels provided, the algorithms find those themselves.
A good example of unsupervised learning is finding clusters in a dataset: you don't need to specify how many clusters there, or what they are named, etc. An unsupervised clustering algorithm will find the clusters in the data without further input. A practical example is grouping photos of the same person on e.g. Google Photos or Facebook.

Example learning algorithms:
- Clustering:
    - k-means
    - Hierarchical Cluster Analysis (HCA)
    - Expected Maximization
- Association rule learning:
    - Aprioiri
    - Eclat 

## k-means

Most of this is from [wikipedia](https://en.wikipedia.org/wiki/K-means_clustering).

k-means clustering aims to partition n observations into k clusters.

Given a set of observations $(x_1, x_2, …, x_n)$, where each observation is a $d$-dimensional real vector, k-means clustering aims to partition the $n$ observations into $k (≤ n)$ sets $S = {S_1, S_2, …, S_k}$ so as to minimize the within-cluster sum of squares (WCSS) (i.e. variance). Formally, the objective is to find:

$${\displaystyle {\underset {\mathbf {S} }{\operatorname {arg\,min} }}\sum _{i=1}^{k}\sum _{\mathbf {x} \in S_{i}}\left\|\mathbf {x} -{\boldsymbol {\mu }}_{i}\right\|^{2}={\underset {\mathbf {S} }{\operatorname {arg\,min} }}\sum _{i=1}^{k}|S_{i}|\operatorname {Var} S_{i}}$$

where $μ_i$ is the mean of points in $S_i$. 

### algorithms
The problem is computationally difficult (NP-hard); however, there are efficient heuristic algorithms that are commonly employed and converge quickly to a local optimum. So these algorithms don't typically find the best (=optimal) global clustering.

# Reinforcement Learning
In reinforement learning, an _agent_ observes the enforment and is able to select and perform action and it get's _rewards_ in return (or _penalties_). It must then learn by itself what the best strategy or policy is.

This approach has had recent successes with e.g. Deepmind's AlphaGo program.

# Batch and Online Learning

In batch learning a system cannot learn incrementally, it needs to have all the data available when it does learning. The system is first trained and then released in production. Typically, batch systems are retrained e.g.: every hour, 24hours, week, etc. This is called **offline learning**.

In online learning, the learning happens simultaneously with outputting values, this is often useful when there is a cotinuous flow of data (e.g. stock prices). An important principle here is that for the *learning rate*: how fast does the system adapt to new data.

# Ensemble Learning

Building a model on top of many other models is called *Ensemble Learning*, and it is often a great way to push ML algorithms even further.

An example is a ```RandomForestRegressor``` (scikit-learn). Random Forests work by training many Decision Trees on random subsets of the features, then averaging out their predictions.